<a href="https://colab.research.google.com/github/kevincallan/emotion_recognition/blob/main/Face_Detection_In_Video_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data analysis packages
import pandas as pd
import numpy as np
# Viz libraries
import matplotlib.pyplot as plt
import seaborn as sns

# loading bar
# fix issue with tqdm and colab
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm_notebook as tqdm

# ***tl:dr***

The purpose of this script was to detect if a face existed in a set of videos and write the result to an analytics base table in order to model relationships at a later stage in the project

 


# Project Background

 

* This was part of my MSc practicum which also happens to be the last non-proprietary analysis I completed this year.

 

* The objective of the project was to understand, extract and create the features that make video sequences memorable.

 

* This particular script extracts the videos from the source files, runs a face detection on each and writes the result into a file to be used as an input into a machine learning model.

 

# How does it work?

 

1. The script iteratively reads each of the videos identified with the file format .webm from the source folder

2. It splits the videos which are 7 seconds in length into keyframes for each second

3. Next it runs the face detector over each frame and writes the result to a file for further analysis

 

# What could be improved?

 

I think there would have been a less hacky and more functional way to write this script that would have included more separate functions for the steps taking rather than having long and harder to read cells. If I was to rewrite this today that is the approach I would take but this way solved the problem I needed to at the time so I was more than happy with the outcome.


In [ ]:
!pip3 install face_recognition
import face_recognition
#!pip3 install skvideo

In [ ]:
!pip install opencv-contrib-python

In [ ]:
# connect information in the google drive to this colab session

from google.colab import drive
drive.mount('/content/drive/')

# Change the file directory to reflect the new folder incluing the sub folder "sources"
# This contains the raw content we need to extract the features from. 

import os
os.chdir("/content/drive/My Drive/Memorability_assignment/")
!ls

In [ ]:
ground_truth_file = 'dev-set/dev-set_ground-truth.csv'
ground_truth = pd.read_csv(ground_truth_file)
ground_truth.head()



1.   for each video in the source folder extract keyframes
2. run face detector on the keyframes 
3. if faces true then write 1 to a text file with the video name as the id 


Progress made: Trying to oextract keyframes from the video, one each second. With these frames run the face detector
Hitting walls while trying to convert the video to and np array. 

Once it's in this array 
Run face detector 
Write results to a txt 
Model difference.




In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

Using TensorFlow backend.


Below takes the video referneced and selects 1 frame per second and saves into the current director 

In [ ]:
#Writes keyframes from one video
#Acts to test code before being run on all the source files
count = 0  
directory = '/content/drive/My Drive/Memorability_assignment/sources/sources/'
frameRate = cap.get(5) #frame rate
x=1
  
for file in os.listdir(directory):
    if file.endswith(".webm"):
        videoFile = file
        cap = cv2.VideoCapture(file) 
        while(cap.isOpened()):
            frameId = cap.get(1) #current frame number
            , frame = cap.read()
            if (ret != True):
                break
            if (frameId % math.floor(frameRate) == 0):
                filename ="/content/drive/My Drive/Memorability_assignment/sources/frame%d.jpg" % count;count+=1
                cv2.imwrite(filename, frame)
                continue
        cap.release()
        print ("Done!")

In [ ]:
#Code that iterates over every file in the directory 
#This code takes a couple of hours to run. 
#Extracting 1 frame per second from each video and placing it in folder
import os
count = 0

directory = '/content/drive/My Drive/Memorability_assignment/sources/sources/'
#if not os.path.exists(folder):
  #  os.makedirs(folder)
for file in os.listdir(directory):
    if file.endswith(".webm"):   # If this is true. Change 
          videoFile = file
          cap = cv2.VideoCapture((os.path.join(directory, file)))   # capturing the video from the given path
          frameRate = cap.get(5) #frame rate
          x=1
          while(cap.isOpened()):
              frameId = cap.get(1) #current frame number
              ret, frame = cap.read()
              if (ret != True):
                  break
              if (frameId % math.floor(frameRate) == 0):                  
                  dirName = '/content/drive/My Drive/Memorability_assignment/sources/%s' % file
                  folder_name = (os.path.splitext(dirName)[0]) # take out the zero to keep extension. Needs to be done to reference the video in the index of model data frame in line with pre existing features 
                  os.makedirs(folder_name, exist_ok=True)  #  Creates the directory but when the loop returns directory already exists 
                  # Next step would be to create directory and build in if directory exists skip
                  # Then write new file into directory. 
                  filename = folder_name+"/frame%d.jpg" % count;count+=1
                 # filename ="/content/drive/My Drive/Memorability_assignment/sources/frame%d.jpg" % count;count+=1
                  #print '%s %d' % (name, number)
                  cv2.imwrite(filename, frame)
          cap.release()
          
          #print(os.path.join(directory, file))
          continue
          print ("Done!")
#     else:
#         continue

In [ ]:
# Testing the process on one image 
#Read one image and return a bool if there is a face in the image

import cv2
import csv

from google.colab.patches import cv2_imshow
# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


# To use a video file as input 
#cap returns a tuple and the frame _, value unpacks the tuple 
img = cv2.imread('/content/drive/My Drive/Memorability_assignment/sources/video3607/frame332.jpg', cv2.IMREAD_COLOR)

frame = img 
        # Convert to grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #removing frame for img
        # Detect the faces
faces = face_cascade.detectMultiScale(gray, 1.3, 4)



face_found = 0   #---Initially set the flag to be False
for (x, y, w, h) in faces:
    if w > 0 :                 #--- Set the flag True if w>0 (i.e, if face is detected)
        face_found = 1
#Extract the video id from the path that the image was in 
vid_id=(path.split('/')[-2]+".webm")

a = [vid_id, face_found]

        
#print (a)
#Write the result of the face detector and its video id to a csv to be modelled

with open('facial_features.csv', 'a') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(a)

       

['video3532.webm', 0]


In [ ]:
#for loop to do the above
#note this will have to iterate through folders 
#entering a video folder, find the resulting file path
import os
from google.colab.patches import cv2_imshow
import os, sys
import shutil
from shutil import copyfile
# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#folder which contains the sub directories
source_dir = '/content/drive/My Drive/Memorability_assignment/sources/'

#list sub directories 
for root, dirs, files in os.walk(source_dir):

#iterate through them
    for i in dirs: 
        #create a new folder with the name of the iterated sub dir
        path = '/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i
        #take random sample, here 3 files per sub dir
        framenames = os.listdir('/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i )     
        for i in framenames:
          frame_path = (os.path.join(path, i))
          print(frame_path)

In [ ]:
#for loop to do the above
#note this will have to iterate through folders 
#entering a video folder, find the resulting file path
#check all the images in the folder and writes the result of the face detector to a csv

#Takes 2+ hours to run. 
import os
from google.colab.patches import cv2_imshow
import os, sys
import shutil
from shutil import copyfile
# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#folder which contains the sub directories
source_dir = '/content/drive/My Drive/Memorability_assignment/sources/'
frame_path = []

#list sub directories 
for root, dirs, files in os.walk(source_dir):

#iterate through them
    for i in dirs: 
        #create a new folder with the name of the iterated sub dir
        path = '/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i
        #take random sample, here 3 files per sub dir
        framenames = os.listdir('/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i )     
        for i in framenames:
          frame_path = [(os.path.join(path, i))]
         # print(frame_path)
          
          for f in frame_path:           
                img = cv2.imread(f, cv2.IMREAD_COLOR)

                frame = img 
                        # Convert to grayscale
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #removing frame for img
                        # Detect the faces
                faces = face_cascade.detectMultiScale(gray, 1.3, 4)



                face_found = 0   #---Initially set the flag to be False
                for (x, y, w, h) in faces:
                    if w > 0 :                 #--- Set the flag True if w>0 (i.e, if face is detected)
                        face_found = 1
                #Extract the video id from the path that the image was in 
                vid_id=(path.split('/')[-2]+".webm")

                a = [vid_id, face_found]
                with open('facial_features.csv', 'a') as csvfile:
                  filewriter = csv.writer(csvfile, delimiter=',',
                                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
                  filewriter.writerow(a)


In [ ]:
df_face = pd.read_csv('facial_features.csv')#, sep='\c', header=None)
df_face.columns = ["video","face_found"]
df_face

In [ ]:
#Testing sub directory iteration 

import os, sys


#folder which contains the sub directories
source_dir = '/content/drive/My Drive/Memorability_assignment/sources/'

#list sub directories 
for root, dirs, files in os.walk(source_dir):

#iterate through them
    for i in dirs: 

        #create a new folder with the name of the iterated sub dir
        path = '/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i
        #take random sample, here 3 files per sub dir
        
        #
        #framepath = '/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i 
        #framenames = os.listdir('/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i )  
        filenames = random.sample(os.listdir('/content/drive/My Drive/Memorability_assignment/sources/' + "%s/" % i ), 7)
        
        #print(framepath)
        #print(path, framenames)
        
        #print (path)
        print (filenames)
 

In [ ]:
df_process_face = df_face.groupby(['video']).mean()
 
# df_explore = pd.merge(ground_truth, df_process_face,left_index=True, right_index=)
# df_explore.head()

df_explore = pd.merge(ground_truth, df_process_face, on=['video','video'])
#df_explore



# df_explore.loc[df_explore.face_found > 0, 'face_found'] = .9
# df_explore.loc[df_explore.face_found == 0, 'face_found'] = 0

df_explore



,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,face_found
0,video10.webm,0.950,34,0.900,10,0.000000
1,video23.webm,0.923,67,0.826,23,0.428571
2,video26.webm,0.863,42,0.636,11,1.000000
3,video66.webm,0.850,45,0.889,18,0.000000
4,video67.webm,0.763,58,0.900,10,0.000000
5,video73.webm,0.706,36,0.769,13,0.857143
6,video100.webm,0.951,33,0.889,9,0.000000
7,video105.webm,0.779,34,0.800,10,0.428571
8,video108.webm,0.950,33,0.889,18,0.142857
9,video121.webm,0.956,34,0.833,12,0.285714
